In [23]:
!pip install pyopengl
!pip install glfw
!pip install pyglm


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import glfw
from OpenGL.GL import *
import OpenGL.GL.shaders
import numpy as np
import glm

In [2]:
# Inicializando uma janela

glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE)
window = glfw.create_window(720, 600, "Programa", None, None)

if (window == None):
  print('Falha em criar janela GLFW')
  glfw.terminate()

glfw.make_context_current(window=window)

In [3]:
vertex_code = """
attribute vec2 position;
uniform mat4 mat_transformation;

void main() {
  gl_Position = mat_transformation * vec4(position, 0.0, 1.0);
}
"""

In [4]:
fragment_code = """
uniform vec4 color;
void main() {
  gl_FragColor = color;
}
"""

In [5]:
# Requisitando slot para GPU
program = glCreateProgram()
vertex = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)

In [6]:
# Associando código aos slots solicitados
glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

In [7]:
# Compilando Vertex shader
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
  error = glGetShaderInfoLog(vertex).decode()
  print(error)
  raise RuntimeError('Erro de compilação de Vertex Shader')

In [8]:
# Compilando Fragment shader
glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
  error = glGetShaderInfoLog(fragment).decode()
  print(error)
  raise RuntimeError('Erro de compilação de Fragment Shader')

In [9]:
# Associando programas compilados ao programa principal
glAttachShader(program, vertex)
glAttachShader(program, fragment)

In [10]:
# Associando ao programa principal
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
  print(glGetProgramInfoLog(program))
  raise RuntimeError('Erro de link ao programa principal')

glUseProgram(program)

In [11]:
# Preparando dados (na CPU) para envio à GPU

vertices = np.zeros(3, [('position', np.float32, 2)])

vertices['position'] = [
  (+0.05, -0.05),
  (+0.05, +0.05),
  (-0.05, -0.05)
]

In [12]:
# Envio de dados à GPU
buffer_VBO = glGenBuffers(1)
glBindBuffer(GL_ARRAY_BUFFER, buffer_VBO)

glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_DYNAMIC_DRAW)
glBindBuffer(GL_ARRAY_BUFFER, buffer_VBO)

In [13]:
# Associando dados ao vertex shader
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)

In [14]:
# Obtendo position, variavel definida no vertex shader
loc = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc)

In [15]:
glVertexAttribPointer(loc, 2, GL_FLOAT, False, stride, offset)

In [16]:
loc_color = glGetUniformLocation(program, 'color')
R = 0.7
G = 0.0
B = 0.2

In [17]:
def multiplica_matriz(a,b):
  m_a = a.reshape(4,4)
  m_b = b.reshape(4,4)
  m_c = np.dot(m_a, m_b)
  c = m_c.reshape(1,16)
  return c

In [18]:
import math

def rotaciona(angulo):
  c = math.cos(angulo)
  s = math.sin(angulo)

  mat_rotation = np.array([
    c, -s, 0.0, 0.0,
    +s, c, 0.0, 0.0,
    0.0, 0.0, 1.0, 0.0,
    0.0, 0.0, 0.0, 1.0 
  ], np.float32)

  return mat_rotation

In [19]:
def escala(sx, sy):
  mat_escala = np.array([
    sx, 0.0, 0.0, 0.0,
    0.0, sy, 0.0, 0.0,
    0.0, 0.0, 1.0, 0.0,
    0.0, 0.0, 0.0, 1.0 
  ])

  return mat_escala

In [20]:
def translada(tx, ty):
  mat_escala = np.array([
    1.0, 0.0, 0.0, tx,
    0.0, 1.0, 0.0, ty,
    0.0, 0.0, 1.0, 0.0,
    0.0, 0.0, 0.0, 1.0 
  ])

  return mat_escala

In [21]:
# Capturando eventos do teclado
def key_event(window, key, scancode, action, mods):
  global t_x, t_y, theta, scale_fac
  
  # translado
  if key == 265: t_y += 0.01 # tecla cima
  if key == 264: t_y -= 0.01 # tecla baixo
  if key == 263: t_x -= 0.01 # tecla direita
  if key == 262: t_x += 0.01 # tecla esquerda

  # rotacao
  if key == 65: theta += 0.1 # tecla a
  if key == 83: theta -= 0.1 # tecla s

  # escala
  if key == 90: scale_fac += 0.01 # tecla z
  if key == 88: scale_fac -= 0.01 # tecla x


glfw.set_key_callback(window, key_event)

In [22]:
glfw.show_window(window)

t_x = 0
t_y = 0
theta = 0
scale_fac = 1

while not glfw.window_should_close(window):
  glClear(GL_COLOR_BUFFER_BIT)
  glClearColor(1.0, 1.0, 1.0, 1.0)

  mat_transform = multiplica_matriz(escala(scale_fac, scale_fac), rotaciona(theta))
  mat_transform = multiplica_matriz(translada(t_x, t_y), mat_transform)

  loc = glGetUniformLocation(program, 'mat_transformation')

  glUniformMatrix4fv(loc, 1, GL_TRUE, mat_transform)
  glDrawArrays(GL_TRIANGLE_STRIP, 0, 4)
  glUniform4f(loc_color, R, G, B, 1.0)

  glfw.swap_buffers(window)
  glfw.poll_events()

glfw.terminate()